In [2]:
import numpy as np
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, RepeatVector, TimeDistributed, Embedding, Dropout, Bidirectional
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Parameters
vocab_size = 10000
max_seq_length = 100  # Maximum length of sequences
embedding_dim = 100   # Size of word embeddings
latent_dim = 64     # Size of the latent space
rnn_cells_first = 256  # Number of cells in the first RNN layer
rnn_cells_second = 128  # Number of cells in the second RNN layer
epochs = 100
batch_size = 256

# Load Reuters dataset
(x_train, _), (x_test, _) = reuters.load_data(num_words=vocab_size)
x_train = pad_sequences(x_train, maxlen=max_seq_length, padding='post')
x_test = pad_sequences(x_test, maxlen=max_seq_length, padding='post')


# Encoder
input_text = Input(shape=(max_seq_length,))
x = Embedding(vocab_size, embedding_dim)(input_text)
x = Bidirectional(LSTM(rnn_cells_first, return_sequences=True))(x)
x = Dropout(0.15)(x)
x = Bidirectional(LSTM(rnn_cells_second))(x)
latent_repr = Dense(latent_dim, activation='relu')(x)
encoder_model = Model(input_text, latent_repr)
encoder_model.summary()

2110848/2110848 [==============================] - 1s 1us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding (Embedding)       (None, 100, 100)          1000000   
                                                                 
 bidirectional (Bidirection  (None, 100, 512)          731136    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 100, 512)          0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 256)               656384    
 onal)                                                           
                                                                

In [3]:

# Decoder
decoder_input = Input(shape=(latent_dim,))
decoder_x = Dense(max_seq_length * embedding_dim, activation='relu')(decoder_input)
decoder_x = RepeatVector(max_seq_length)(decoder_x)
decoder_x = LSTM(rnn_cells_second, return_sequences=True)(decoder_x)
decoder_x = Dropout(0.15)(decoder_x)
decoder_x = LSTM(rnn_cells_first, return_sequences=True)(decoder_x)
decoder_output = TimeDistributed(Dense(vocab_size, activation='softmax'))(decoder_x)
decoder_model = Model(decoder_input, decoder_output)
decoder_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64)]              0         
                                                                 
 dense_1 (Dense)             (None, 10000)             650000    
                                                                 
 repeat_vector (RepeatVecto  (None, 100, 10000)        0         
 r)                                                              
                                                                 
 lstm_2 (LSTM)               (None, 100, 128)          5186048   
                                                                 
 dropout_1 (Dropout)         (None, 100, 128)          0         
                                                                 
 lstm_3 (LSTM)               (None, 100, 256)          394240    
                                                           

In [4]:

# Autoencoder
autoencoder = Model(input_text, decoder_model(encoder_model(input_text)))
autoencoder.compile(optimizer=Adam(learning_rate=0.0005), loss='sparse_categorical_crossentropy')
autoencoder.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 model (Functional)          (None, 64)                2403968   
                                                                 
 model_1 (Functional)        (None, 100, 10000)        8800288   
                                                                 
Total params: 11204256 (42.74 MB)
Trainable params: 11204256 (42.74 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:

from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow.keras.backend as K

def custom_lr_scheduler(epoch, lr):
    # Decrease learning rate by 0.1 factor every 5 epochs
    if epoch % 5 == 0 and epoch != 0:
        lr = lr * 0.1
    return lr

# Define the callback
lr_scheduler = LearningRateScheduler(custom_lr_scheduler)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
autoencoder.fit(x_train, x_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, x_test), callbacks=[early_stopping, lr_scheduler])


Epoch 1/100
36/36 [==============================] - 23s 310ms/step - loss: 7.4231 - val_loss: 5.7901 - lr: 5.0000e-04
Epoch 2/100
36/36 [==============================] - 8s 235ms/step - loss: 5.7929 - val_loss: 5.7549 - lr: 5.0000e-04
Epoch 3/100
36/36 [==============================] - 7s 199ms/step - loss: 5.7625 - val_loss: 5.7419 - lr: 5.0000e-04
Epoch 4/100
36/36 [==============================] - 6s 178ms/step - loss: 5.7400 - val_loss: 5.7107 - lr: 5.0000e-04
Epoch 5/100
36/36 [==============================] - 8s 215ms/step - loss: 5.6983 - val_loss: 5.6966 - lr: 5.0000e-04
Epoch 6/100
36/36 [==============================] - 8s 212ms/step - loss: 5.6634 - val_loss: 5.6493 - lr: 5.0000e-05
Epoch 7/100
36/36 [==============================] - 7s 194ms/step - loss: 5.6517 - val_loss: 5.6439 - lr: 5.0000e-05
Epoch 8/100
36/36 [==============================] - 6s 182ms/step - loss: 5.6463 - val_loss: 5.6396 - lr: 5.0000e-05
Epoch 9/100
36/36 [==============================] - 6s

In [9]:
# Encode the input text using the encoder model
encoded_texts = encoder_model.predict(x_test[:10])

# Decode the encoded texts using the decoder model
# This gives you a probability distribution for each word in the sequence
prob_distributions = decoder_model.predict(encoded_texts)


1/1 [==============================] - 0s 23ms/step


In [10]:
def sample(preds, temperature=1.0):
    # Convert to array and prevent numerical issues with very small numbers
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-7) / temperature  # Adjust by temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)  # Softmax
    probas = np.random.multinomial(1, preds, 1)  # Sample from the softmax distribution
    return np.argmax(probas)


In [14]:
word_index = reuters.get_word_index()
reverse_word_index = {value: key for (key, value) in word_index.items()}
def decode_sequence_with_sampling(prob_distributions, temperature=1.0):
    return ' '.join([reverse_word_index.get(sample(probs, temperature) - 3, '?') for probs in prob_distributions])

# Example usage with a specified temperature
temperature = 1  # Adjust this value to change randomness
decoded_texts = [decode_sequence_with_sampling(seq, temperature) for seq in prob_distributions]
for text in decoded_texts:
    print('\n')
    print(f'Decoded: {text}')
    print('\n')




Decoded: regulators non further has the 0 plant preliminary at 70 operations to ? in reuter president fixed this month ? substantial ? 600 of to stabilize ? a warrant 000 in by ? ? ? in 100 ? to ? expired ? net on dlrs ? york ? 8 ? asset just ? ? it ? sheets s standard marks if contents ? ? ? ? ? ? 95 15 dlrs cts to intends ? ? vs ? ? heavy pct bringing ? in its of reuter ? new group's costs in hours by ? vs billion ? ? is




Decoded: ? ? normal tonnes ? among billion guarantee year ? ? ? dairy to too business national about kingdom file co breaking vs ? 821 inc 839 proposal stage banking finding mln dlrs ? again the after ? 380 for demand ? april well revs 3 reduced 1 six 7 ? reached agreement texas and reuter because fed of ? with 4 ? ? 3 strongly on ? ? sparked ccc ? since of ? ? high the ? which ? pct ? 041 likely the prior 30 than ? ? dlrs connection ? ? second ? the dlrs industry




Decoded: meeting will fund is staff sb 3 two one performing vs 31 the receipts ? april rise of